## Prepare RAFS API Client

In [ ]:
import json
import os
import sys
from pprint import pprint

from dotenv import dotenv_values

sys.path.append("..")

from client.api.settings import ApiClientSettings 
from client.api_client import APIClient
from utils import print_json, load_wpc_payload

conf = dotenv_values("./demo.env")

api_client = APIClient(
    conf["osdu_host"],
    conf["version"],
    conf["url_prefix"],
    conf["data_partition_id"],
    conf["access_token"],
)


## Load MasterData and WorkProductComponent Payloads

In [ ]:
master_data = load_wpc_payload("md_payloads/master_data.json", conf)

print_json(master_data[0]["id"])
print_json(master_data[1]["id"])
print_json(master_data[2]["id"])
print_json(master_data[3]["id"])
print_json(master_data[4]["id"])
print_json(master_data[5]["id"])

In [ ]:
samples_analyses_report_wpc = load_wpc_payload("wpc_payloads/samples_analyses_report.json", conf)

print_json(samples_analyses_report_wpc[0]["id"])

In [ ]:
samples_analysis = load_wpc_payload("wpc_payloads/samples_analysis.json", conf)

print_json(samples_analysis[0]["id"])
print_json(samples_analysis[1]["id"])
print_json(samples_analysis[2]["id"])
print_json(samples_analysis[3]["id"])

## POST MasterData

In [ ]:
md_response = api_client.post("/masterdata", json=master_data)

pprint(md_response.json())

## POST SamplesAnalysesReport

In [ ]:
sar_response = api_client.post("/samplesanalysesreport", json=samples_analyses_report_wpc)

pprint(sar_response.json())

## POST SamplesAnalysis

In [ ]:
sa_response = api_client.post("/samplesanalysis", json=samples_analysis)

pprint(sa_response.json())

## Build & display parquet data

In [ ]:
from utils import display_parquet
from utils import build_parquet_from_json

# Json and Parquet Paths
gcms_alkanes_json_path = "content_payloads/gcms_alkanes_orient_split.json"
multistageseparator_json_path = "content_payloads/multistageseparator_orient_split.json"
multiple_salinity_json_path = "content_payloads/multiple_salinity_orient_split.json"
nmr_json_path = "content_payloads/nmr_orient_split.json"

gcms_alkanes_parquet_path = "content_payloads/gcms_alkanes.parquet"
multistageseparator_parquet_path = "content_payloads/multistageseparator.parquet"
multiple_salinity_parquet_path = "content_payloads/multiple_salinity.parquet"
nmr_parquet_path = "content_payloads/nmr.parquet"

# Build parquets
build_parquet_from_json(gcms_alkanes_json_path, gcms_alkanes_parquet_path, conf["data_partition_id"])
build_parquet_from_json(multistageseparator_json_path, multistageseparator_parquet_path, conf["data_partition_id"])
build_parquet_from_json(multiple_salinity_json_path, multiple_salinity_parquet_path, conf["data_partition_id"])
build_parquet_from_json(nmr_json_path, nmr_parquet_path, conf["data_partition_id"])


## Post & Get MultiStageSeparator test data (Parquet format)

In [ ]:
from utils import get_parquet_as_binary

mss_wpc_id = f"{conf['data_partition_id']}:work-product-component--SamplesAnalysis:MultiStageSeparatorTest_WPC"
parquet_headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/x-parquet"}

mss_data_response = api_client.post(
    f"/samplesanalysis/{mss_wpc_id}/data/multistageseparator",
    data=get_parquet_as_binary(multistageseparator_parquet_path), 
    headers=parquet_headers)

pprint(mss_data_response.json())
mss_dataset_id = mss_data_response.json()["ddms_urn"].split("/")[-2]
print(mss_dataset_id)

In [ ]:
mss_get_data_response = api_client.get(
    f"/samplesanalysis/{mss_wpc_id}/data/multistageseparator/{mss_dataset_id}", 
    headers=parquet_headers)

print(mss_get_data_response.content[:300], "...")

### Get MultiStageSeparator test (Json format)

In [ ]:
json_headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

mss_get_data_response = api_client.get(
    f"/samplesanalysis/{mss_wpc_id}/data/multistageseparator/{mss_dataset_id}", 
    headers=json_headers)

print_json(mss_get_data_response.json())

## Update Parquet Example (GCMS Alkanes)

In [ ]:
from utils import update_parquet, get_parquet_cell_value

# Update SamplesAnalysisID
update_parquet(
    path=gcms_alkanes_parquet_path,
    row_index=0,
    column_name="SamplesAnalysisID",
    new_value=f"{conf['data_partition_id']}:work-product-component--SamplesAnalysis:GCMS_Alkanes_WPC:",
)

## Post & Get GCMS_Alkanes data (Parquet format)

In [ ]:
from utils import get_parquet_as_binary

gcms_alkanes_wpc_id = f"{conf['data_partition_id']}:work-product-component--SamplesAnalysis:GCMS_Alkanes_WPC"
parquet_headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/x-parquet"}

gcms_alkanes_data_response = api_client.post(
    f"/samplesanalysis/{gcms_alkanes_wpc_id}/data/gcmsalkanes",
    data=get_parquet_as_binary(gcms_alkanes_parquet_path), 
    headers=parquet_headers)

pprint(gcms_alkanes_data_response.json())
gcms_alkanes_dataset_id = gcms_alkanes_data_response.json()["ddms_urn"].split("/")[-2]
print(gcms_alkanes_dataset_id)

In [ ]:
gcms_alkanes_get_data_response = api_client.get(
    f"/samplesanalysis/{gcms_alkanes_wpc_id}/data/gcmsalkanes/{gcms_alkanes_dataset_id}", 
    headers=parquet_headers)

print(gcms_alkanes_get_data_response.content[:300], "...")

## Get GCMS Alkanes (Json format)

In [ ]:
gcms_alkanes_wpc_id = f"{conf['data_partition_id']}:work-product-component--SamplesAnalysis:GCMS_Alkanes_WPC"
json_headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

gcms_alkanes_get_data_response = api_client.get(
    f"/samplesanalysis/{gcms_alkanes_wpc_id}/data/gcmsalkanes/{gcms_alkanes_dataset_id}", 
    headers=json_headers)

print_json(gcms_alkanes_get_data_response.json())

## Post & Get NMR test (Parquet format)

In [ ]:
from utils import get_parquet_as_binary

nmr_wpc_id = f"{conf['data_partition_id']}:work-product-component--SamplesAnalysis:NMR_Test_WPC"
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/x-parquet"}

nmr_data_response = api_client.post(
    f"/samplesanalysis/{nmr_wpc_id}/data/nmr",
    data=get_parquet_as_binary(nmr_parquet_path), 
    headers=headers)

pprint(nmr_data_response.json())
nmr_dataset_id = nmr_data_response.json()["ddms_urn"].split("/")[-2]
print(nmr_dataset_id)

In [ ]:
nmr_data_response = api_client.get(
    f"/samplesanalysis/{nmr_wpc_id}/data/nmrtests/{nmr_dataset_id}", 
    headers=headers)

print(nmr_data_response.content[:300], "...")

## Get NMR test (Json format)

In [ ]:
json_headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

nmr_get_data_response = api_client.get(
    f"/samplesanalysis/{nmr_wpc_id}/data/nmr/{nmr_dataset_id}", 
    headers=json_headers)

print_json(nmr_get_data_response.json())

### Post & Get MultipleSalinity data (Json Format)

In [ ]:
import json

with open("content_payloads/multiple_salinity_orient_split.json") as fp:
    multiple_salinity_test_json_data = json.loads(fp.read().replace("{data_partition_id}", conf["data_partition_id"]))
    

multiple_salinity_wpc_id = f"{conf['data_partition_id']}:work-product-component--SamplesAnalysis:MultipleSalinity_Test_WPC"
accept_header = {"Accept": "*/*;version=1.0.0"}
multiple_salinity_data_response = api_client.post(
    f"/samplesanalysis/{multiple_salinity_wpc_id}/data/multiplesalinitytests",
    json=multiple_salinity_test_json_data, 
    headers=accept_header)

pprint(multiple_salinity_data_response.json())
multiple_salinity_dataset_id = multiple_salinity_data_response.json()["ddms_urn"].split("/")[-2]
print(multiple_salinity_dataset_id)

In [ ]:
multiple_salinity_data_response = api_client.get(
    f"/samplesanalysis/{multiple_salinity_wpc_id}/data/multiplesalinitytests/{multiple_salinity_dataset_id}", 
    headers=accept_header)

print_json(multiple_salinity_data_response.json())